In [56]:
# bringing in the necessary libraries

import urllib3
import re
import json
import pyodbc

# since my machine (I think) lacks the proper certifications for this project,
# the next line of code in conjunction with cert_reqs='CERT_NONE' temporarily bandades error issues.

urllib3.disable_warnings()

In [57]:
# declaring sensitive variables provided in separate document.

with open('C:\\Users\\Buyer\\Desktop\\School\\DA 320\\Credentials\\credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

In [59]:
# connecting to the actual database

conn = pyodbc.connect("Driver={SQL Server};"
                      "Server=THEGARDEN\MSSQLSERVER1;"
                      "Database=PortfolioProject;"
                      "Trusted_Connection=Yes;"
                      "UID=" + username + ";"
                      "PWD=" + password)

In [60]:
# declaring an identity variable the url server requires.

headers = {"User-Agent": "Chrome/94.0.4606.71"}

# now we're gonna party like it's 2001

url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2001&sort=desc&view=detailed'

In [61]:
# https://stackoverflow.com/questions/34735761/https-url-ignore-cert-warning-and-continue?rq=1
# the link above helped me discover how to work around the SSL Certificate error with urllib3.

http = urllib3.PoolManager(cert_reqs='CERT_NONE')

# initiating a request to check out the url

r = http.request('GET', url, headers=headers)

# changing to data type to make the request readable

text = str(r.data, 'utf-8')

In [62]:
# declaring RegEx variables and where to search for them.

titles = re.findall('class="title"><h3>(.*)<\/h3>', text)
dates = re.findall('<span>(.*)<\/span>\n\n', text)
summaries = re.findall('<div class="summary">\n\s+(.*)', text)
scores = re.findall('>(\d*)<\/div>\n\s+<\/a>\n\n\s+<\/div>\n\n', text)
images = re.findall('movie\/.*><img src="(.*.jpg)', text)

#an empty array to place all of the data into.

array = []

In [63]:
# declaring a cursor object to use later
# as well as the SQL+Python code and column names.

cursor = conn.cursor()
sql = "INSERT INTO MetaCriticMovies (MovieTitle, ReleaseDate, Description, Metascore, ThumbnailImage) values (?,?,?,?,?)"
args = "Title", "Release Date", "Summary", "Score", "ImgURL"


In [71]:
# Another David Arvilla inspired loop
# For each movie title, print the findings of each RegEx variable

for i in range(len(titles)):
    args = titles[i], dates[i], summaries[i], scores[i], images[i]
    cursor.execute(sql, args)
    cursor.commit()

In [135]:
# Inspired by David Arvilla to put everything into an array
# and then print out the contents through a loop.

#for i in range(100):
#    array.append(str(i+1) + ") " +
#                     str("Title: " + titles[i] + 
#                     " || Release Date: " + dates[i] +
#                     " || Metascore: " + scores[i] + "\n"
#                     "Summary: " + summaries[i] + "\n"
#                     "Image: " + images[i]))

In [25]:
# If we wanted to just print everything here:

# for as many times as the array is long...
# print out each part followed by two lines.

#for i in range(len(array)):
#    print(str(array[i]), "\n\n")